# 第二章第四节：时序逻辑

## 本节的目的

没有状态，您不可能能写任何有意义的数字逻辑。没有状态，您不可能能写任何有意义的数字逻辑。没有状态，...

重要的事情重复三次。因为如果不能存储中间结果，您做不了多少事情。

好吧，把上面这个糟糕的笑话放一边，这个模块将描述如何在Chisel中表达常见的时序逻辑。在模块结束时，您应该能够在Chisel中实现和测试一个移位寄存器的模块。

重要的是要强调这一部分可能不会让您惊叹Chisel的能力。Chisel的力量不在于新的时序逻辑模式，而在于设计的参数化。在我们展示这种能力之前，我们必须了解时序逻辑是什么。因此，本节将向您展示Chisel几乎可以完成所有Verilog所能做的事情 - 您只需要学习Chisel语法即可。

## 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# 寄存器
Chisel中的具有状态的基本单元是寄存器，表示为`Reg`。
`Reg`会一直保持其输出值，直到时钟的上升沿，它才接受输入值。
默认情况下，每个Chisel的模块`Module`都有一个隐藏的时钟信号，设计中的每个寄存器都使用这个时钟信号。
这样可以避免在代码到处都指定时钟信号。

<span style="color:blue">**例子：使用寄存器**</span><br>
下面的代码块实现了一个模块，该模块接受输入，加1，并将其作为寄存器的输入连接。
*注意：对于多时钟信号的设计，这个隐藏的时钟信号可以被显示的指定。相关示例，请参阅附录。*

In [3]:
class RegisterModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  val register = Reg(UInt(12.W))
  register := io.in + 1.U
  io.out := register
}

class RegisterModuleTester(c: RegisterModule) extends PeekPokeTester(c) {
  for (i <- 0 until 100) {
    poke(c.io.in, i)
    step(1)
    expect(c.io.out, i+1)
  }
}
assert(chisel3.iotesters.Driver(() => new RegisterModule) { c => new RegisterModuleTester(c) })
println("SUCCESS!!")

[info] [0.001] Elaborating design...
[info] [0.638] Done elaborating.
Total FIRRTL Compile Time: 406.4 ms
Total FIRRTL Compile Time: 16.5 ms
End of dependency graph
Circuit state created
[info] [0.002] SEED 1562377773901
test cmd2HelperRegisterModule Success: 100 tests passed in 105 cycles taking 0.070775 seconds
[info] [0.058] RAN 100 CYCLES PASSED
SUCCESS!!


defined class RegisterModule
defined class RegisterModuleTester

寄存器是通过调用`Reg(tpe)`创建的，其中`tpe`是一个Chisel类型。
在这个例子中，`tpe`是一个12位的`UInt`。

看看上面的测试是如何工作的。
在`poke()`和`expect`的调用之间，中间有一个`step(1)`的调用。
这是告诉测试工具将时钟往前走一个周期，这将导致寄存器的输入传递到输出。

调用`step(n)`将往前走`n`个时钟周期。

读者可能会注意到这一节之前的测试（用于组合逻辑）没有调用`step()`。这是因为在输入上调用`poke()`会立即通过组合逻辑传递最新的值。而如果需要在时序逻辑中更新状态的话，就需要调用`step()`。

下面的代码块会显示上面的`RegisterModule`模块所生成的verilog。

注意：
* 模块有一个您并没有添加的时钟（和复位）输入 - 这是隐藏的时钟信号
* 变量`register`和预期中一样显示为`reg [11:0]`
* 有一块被`ifdef Randomize`分隔开来的部分，在仿真开始之前会将寄存器初始化为一些随机值
* `register`在`posedge clock`（时钟的上升沿）上更新

In [4]:
println(getVerilog(new RegisterModule))

[info] [0.000] Elaborating design...
[info] [0.003] Done elaborating.
Total FIRRTL Compile Time: 160.1 ms
module cmd2HelperRegisterModule(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
  reg [11:0] register; // @[cmd2.sc 7:21]
  reg [31:0] _RAND_0;
  assign io_out = register; // @[cmd2.sc 9:10]
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
      `else
        #0.002 begin end
      `endif
    `endif
  `ifdef RANDOMIZE_REG_INIT
  _RAND_0 = {1{`RANDOM}};
  register = _RAND_0[11:0];
  `endif // RANDOMIZE_REG_

需要注意的是，Chisel会区分类型（如`UInt`）和硬件节点（如 literal`2.U`，或`myReg`的输出）。而
```scala
val myReg = Reg(UInt(2.W))
```

是合法的，因为`Reg`需要使用数据类型作为模板，而
```scala
val myReg = Reg(2.U)
```

是错误的，因为`2.U`已经是一个硬件节点，不能用作模板。

<span style="color:blue">**例子：RegNext**</span><br>
Chisel提供了一个便捷的寄存器对象`RegNext`，用于具有简单输入的寄存器。之前的模块可以缩短为下面这样。注意我们不需要指定寄存器位宽，Chisel可以从寄存器的输出连接中推断出来，在这里是`io.out`。

In [5]:
class RegNextModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  // register bitwidth is inferred from io.out
  io.out := RegNext(io.in + 1.U)
}

class RegNextModuleTester(c: RegNextModule) extends PeekPokeTester(c) {
  for (i <- 0 until 100) {
    poke(c.io.in, i)
    step(1)
    expect(c.io.out, i+1)
  }
}
assert(chisel3.iotesters.Driver(() => new RegNextModule) { c => new RegNextModuleTester(c) })
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.006] Done elaborating.
Total FIRRTL Compile Time: 32.6 ms
Total FIRRTL Compile Time: 22.5 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1562377776476
test cmd4HelperRegNextModule Success: 100 tests passed in 105 cycles taking 0.035812 seconds
[info] [0.035] RAN 100 CYCLES PASSED
SUCCESS!!


defined class RegNextModule
defined class RegNextModuleTester

除了寄存器的名字是自动生成的而不是显式指定的之外，Verilog看起来与之前几乎相同。

In [6]:
println(getVerilog(new RegNextModule))

[info] [0.000] Elaborating design...
[info] [0.004] Done elaborating.
Total FIRRTL Compile Time: 43.0 ms
module cmd4HelperRegNextModule(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
  reg [11:0] _T_2; // @[cmd4.sc 8:20]
  reg [31:0] _RAND_0;
  assign io_out = _T_2; // @[cmd4.sc 8:10]
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
      `else
        #0.002 begin end
      `endif
    `endif
  `ifdef RANDOMIZE_REG_INIT
  _RAND_0 = {1{`RANDOM}};
  _T_2 = _RAND_0[11:0];
  `endif // RANDOMIZE_REG_INIT
  `endif 

---
# `RegInit`

`RegisterModule`中的寄存器被初始化为随机数据以进行仿真。
除非特别指定，否则寄存器没有复位值（或复位，reset value）。
创建具有复位值的寄存器的是使用`RegInit`。

例如，下面创建初始值为零的12位寄存器。
以下两个版本都是有效的，并执行相同的操作：
```scala
val myReg = RegInit(UInt(12.W), 0.U)
val myReg = RegInit(0.U(12.W))
```


第一个版本有两个参数。
第一个参数指定数据类型及其宽度。
第二个参数是一个硬件节点，用来指定复位值，在本例中为0。

第二个版本有一个参数。
它是指定复位值的硬件节点，但通常为“0.U”。

<span style="color:blue">**例子：初始化寄存器** </span><br>
下面演示了怎样使用`RegInit()`，初始化为零。

In [7]:
class RegInitModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  val register = RegInit(0.U(12.W))
  register := io.in + 1.U
  io.out := register
}

println(getVerilog(new RegInitModule))

[info] [0.000] Elaborating design...
[info] [0.009] Done elaborating.
Total FIRRTL Compile Time: 26.6 ms
module cmd6HelperRegInitModule(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
  reg [11:0] register; // @[cmd6.sc 7:25]
  reg [31:0] _RAND_0;
  wire [11:0] _T_1; // @[cmd6.sc 8:21]
  assign _T_1 = io_in + 12'h1; // @[cmd6.sc 8:21]
  assign io_out = register; // @[cmd6.sc 9:10]
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
      `else
        #0.002 begin end
      `endif
    `endif
  `ifdef RANDOMIZE_RE

defined class RegInitModule

请注意，生成的verilog现在有一个代码块，用于检查`if (reset)`以便将寄存器复位为0。
另外还需要注意的是，这是在`always @(posedge clock)`块中。
Chisel的隐含的重置是在时钟的上升沿，并且是同步的。
在真正重置之前，寄存器里仍然是随机的垃圾值。
`PeekPokeTesters`在运行测试之前总是调用reset，但您也可以使用`reset(n)`函数来手动reset，其中`n`表示reset信号会持续为`n`个周期。

---
# 控制流

在控制流方面，寄存器与线非常类似。
它们也具有最后的连接语义，也可以通过`when`，`elsewhen`和`otherwise`根据条件来赋值。

<span style="color:blue">**例子：包含寄存器的控制流**</span><br>
下面的例子使用条件对寄存器赋值，从而得到输入序列中的最大值。

In [8]:
class FindMax extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(10.W))
    val max = Output(UInt(10.W))
  })

  val max = RegInit(0.U(10.W))
  when (io.in > max) {
    max := io.in
  }
  io.max := max
}
assert(chisel3.iotesters.Driver(() => new FindMax) {
  c => new PeekPokeTester(c) {
    expect(c.io.max, 0)
    poke(c.io.in, 1)
    step(1)
    expect(c.io.max, 1)
    poke(c.io.in, 3)
    step(1)
    expect(c.io.max, 3)
    poke(c.io.in, 2)
    step(1)
    expect(c.io.max, 3)
    poke(c.io.in, 24)
    step(1)
    expect(c.io.max, 24)
  }
})
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.088] Done elaborating.
Total FIRRTL Compile Time: 20.3 ms
Total FIRRTL Compile Time: 14.0 ms
End of dependency graph
Circuit state created
[info] [0.001] SEED 1562377777544
test cmd7HelperFindMax Success: 5 tests passed in 9 cycles taking 0.007961 seconds
[info] [0.008] RAN 4 CYCLES PASSED
SUCCESS!!


defined class FindMax

---
# 其他寄存器的例子

寄存器上所执行的操作实际上是在寄存器的**输出**上执行的，具体的操作取决于寄存器的类型。
这意味着您可以写：
```scala
val reg: UInt = Reg(UInt(4.W))```
这也意味着常量`reg`是`UInt`类型的，`UInt`的运算，如`+`，`-`等，对`reg`也成立。

您也不限于使用`UInt`作为寄存器类型，任何基类为`chisel3.Data`的子类都可以。这包括用于有符号整数`SInt`和许多其他类型。

<span style="color:blue">**例子：梳状滤波器**</span><br>
下面是一个梳状滤波器的例子：

In [9]:
class Comb extends Module {
  val io = IO(new Bundle {
    val in  = Input(SInt(12.W))
    val out = Output(SInt(12.W))
  })

  val delay: SInt = Reg(SInt(12.W))
  delay := io.in
  io.out := io.in - delay
}
println(getVerilog(new Comb))

[info] [0.000] Elaborating design...
[info] [0.009] Done elaborating.
Total FIRRTL Compile Time: 39.4 ms
module cmd8HelperComb(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
  reg [11:0] delay; // @[cmd8.sc 7:24]
  reg [31:0] _RAND_0;
  wire [11:0] _T_1; // @[cmd8.sc 9:19]
  assign _T_1 = $signed(io_in) - $signed(delay); // @[cmd8.sc 9:19]
  assign io_out = $signed(_T_1); // @[cmd8.sc 9:10]
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
      `else
        #0.002 begin end
      `endif
    `endif
  `ifdef R

defined class Comb

---
# 练习
<span style="color:red">**练习：移位寄存器**</span><br>

基于您刚刚学习的寄存器的知识，构建一个LFSR的移位寄存器模块。规格如下：
 - 每个元素都是1比特宽。
 - 输出信号有4比特。
 - 输入也是1比特，这就是送入移位寄存器的下一个值。
 - 将移位寄存器的值并行输出，最高位是移位寄存器的最后一个元素，最低位是移位寄存器的第一个元素。 `Cat`函数可能会派上用场。
 -  **输出的初始值为`b0001`。**
 - 每个时钟周期移位（没有使能enable信号）。
 - **注意在Chisel中，给subword赋值是非法的**；例如`out(0) := in`非法。

<img src="images/shifter4.svg" alt="shift register figure" style="width: 450px" />

下面提供了基本的模块框架，测试向量和驱动。第一个寄存器已经为您提供好了：

In [ ]:
class MyShiftRegister(val init: Int = 1) extends Module {
  val io = IO(new Bundle {
    val in  = Input(Bool())
    val out = Output(UInt(4.W))
  })

  val state = RegInit(UInt(4.W), init.U)

  ???
}

class MyShiftRegisterTester(c: MyShiftRegister) extends PeekPokeTester(c) {
  var state = c.init
  for (i <- 0 until 10) {
    // poke in LSB of i (i % 2)
    poke(c.io.in, i % 2)
    // update expected state
    state = ((state * 2) + (i % 2)) & 0xf
    step(1)
    expect(c.io.out, state)
  }
}
assert(chisel3.iotesters.Driver(() => new MyShiftRegister()) {
    c => new MyShiftRegisterTester(c)
})
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>答案</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val nextState = (state << 1) | io.in
  state := nextState
  io.out := state
</pre></article></div></section></div>

<span style="color:red">**练习：参数化的移位寄存器（可选）**</span><br>
写一个移位寄存器，它具有两个参数：延迟（`n`）和初始值（`init`）。并且它还有一个使能的输入信号（`en`）。

In [ ]:
// n 是输出信号的宽度 (delay的数目 - 1)
// 初始值为init
class MyOptionalShiftRegister(val n: Int, val init: BigInt = 1) extends Module {
  val io = IO(new Bundle {
    val en  = Input(Bool())
    val in  = Input(Bool())
    val out = Output(UInt(n.W))
  })

  val state = RegInit(init.U(n.W))

  ???
}

class MyOptionalShiftRegisterTester(c: MyOptionalShiftRegister) extends PeekPokeTester(c) {
  val inSeq = Seq(0, 1, 1, 1, 0, 1, 1, 0, 0, 1)
  var state = c.init
  var i = 0
  poke(c.io.en, 1)
  while (i < 10 * c.n) {
    // 重复送入信号 inSeq
    val toPoke = inSeq(i % inSeq.length)
    poke(c.io.in, toPoke)
    // 更新期望的状态
    state = ((state * 2) + toPoke) & BigInt("1"*c.n, 2)
    step(1)
    expect(c.io.out, state)

    i += 1
  }
}

// 测试不同的深度
for (i <- Seq(3, 4, 8, 24, 65)) {
  println(s"Testing n=$i")
  assert(chisel3.iotesters.Driver(() => new MyOptionalShiftRegister(n = i)) {
    c => new MyOptionalShiftRegisterTester(c)
  })
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>例子</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val nextState = (state << 1) | io.in
  when (io.en) {
    state  := nextState
  }
  io.out := state
</pre></article></div></section></div>

---
# 附录：显式的时钟信号和复位信号

Chisel模块具有默认的时钟和复位信号，在模块中创建的每个寄存器都隐式使用它们。
有时您可能不想要使用这种默认的信号；也许您有一个黑盒可以生成时钟或复位信号，或者您有一个多时钟设计。

Chisel提供了处理这种情况的结构。
时钟和复位信号可以通过使用`withClock（）{}`，`withReset（）{}`以及`withClockAndReset（）{}`，分别或一起被override。
下面的代码块将给出使用这些函数的示例。

需要注意的一点是`reset`（至少在本教程编写时）始终是同步的并且类型为`Bool`。
时钟信号在Chisel（`Clock`）中有自己的类型，应该这样声明。
*`Bool`可以通过调用 `asClock()`来转换为`Clock`类型，但是您应该小心谨慎*

另外要注意的是，`chisel-testers`目前还没有完全支持多时钟设计。

<span style="color:blue">**例子：多时钟模块**</span><br>
一个具有多个时钟和复位信号的模块。

In [11]:
// 我们需要导入多时钟的功能
import chisel3.experimental.{withClock, withReset, withClockAndReset}

class ClockExamples extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(10.W))
    val alternateReset    = Input(Bool())
    val alternateClock    = Input(Clock())
    val outImplicit       = Output(UInt())
    val outAlternateReset = Output(UInt())
    val outAlternateClock = Output(UInt())
    val outAlternateBoth  = Output(UInt())
  })

  val imp = RegInit(0.U(10.W))
  imp := io.in
  io.outImplicit := imp

  withReset(io.alternateReset) {
    //此范围内都使用 alternateReset 作为复位信号
    val altRst = RegInit(0.U(10.W))
    altRst := io.in
    io.outAlternateReset := altRst
  }

  withClock(io.alternateClock) {
    val altClk = RegInit(0.U(10.W))
    altClk := io.in
    io.outAlternateClock := altClk
  }

  withClockAndReset(io.alternateClock, io.alternateReset) {
    val alt = RegInit(0.U(10.W))
    alt := io.in
    io.outAlternateBoth := alt
  }
}

println(getVerilog(new ClockExamples))

[info] [0.000] Elaborating design...
[info] [0.018] Done elaborating.
Total FIRRTL Compile Time: 31.0 ms
module cmd10HelperClockExamples(
  input        clock,
  input        reset,
  input  [9:0] io_in,
  input        io_alternateReset,
  input        io_alternateClock,
  output [9:0] io_outImplicit,
  output [9:0] io_outAlternateReset,
  output [9:0] io_outAlternateClock,
  output [9:0] io_outAlternateBoth
);
  reg [9:0] imp; // @[cmd10.sc 14:20]
  reg [31:0] _RAND_0;
  reg [9:0] _T; // @[cmd10.sc 20:25]
  reg [31:0] _RAND_1;
  reg [9:0] _T_1; // @[cmd10.sc 26:25]
  reg [31:0] _RAND_2;
  reg [9:0] _T_2; // @[cmd10.sc 32:22]
  reg [31:0] _RAND_3;
  assign io_outImplicit = imp; // @[cmd10.sc 16:18]
  assign io_outAlternateReset = _T; // @[cmd10.sc 22:26]
  assign io_outAlternateClock = _T_1; // @[cmd10.sc 28:26]
  assign io_outAlternateBoth = _T_2; // @[cmd10.sc 34:25]
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifd

import chisel3.experimental.{withClock, withReset, withClockAndReset}


defined class ClockExamples

---
# 总结

恭喜您完成了这一节!!您现在已经学会了如何在Chisel中创建寄存器和实现时序逻辑，这意味着您有足够的知识开始来编写真正的电路了。

下一节中将把我们学到的所有内容合并到一个例子中！如果您需要更多的鼓励，请记住下面Chisel专家的这些话：

![BobRoss](http://i.qkme.me/3qbd5u.jpg)

---
# 本节结束!
​
[返回顶部](#top)